In [1]:
# 引用設定
import variable

# 服務名稱
bucketName = variable.bucketName
queueName = variable.queueName
tableName = variable.tableName

# 服務連線
s3_resource = variable.s3_resource
sqs_client = variable.sqs_client
dynamoDB_resource = variable.dynamoDB_resource

# 使用的參數
# http://s3.vcloudlab.pro:4569/S3/vcloudlab_bucket/
baseUrl = variable.baseURL

In [2]:
# 創建 s3 Bucket
s3_resource.create_bucket( Bucket = bucketName )

# 創建 sqs Queue
sqs_client.create_queue( QueueName = queueName )

# 創建 DynamoDB Table
vlabTable = dynamoDB_resource.create_table(
    TableName = tableName,
    
    KeySchema = [
      {
          'AttributeName': 'id',
          'KeyType': 'HASH'
      },
        
    ],
    
    AttributeDefinitions = [
      {
          'AttributeName': 'id',
          'AttributeType': 'S'
      },
    ],
    
    ProvisionedThroughput = {
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)
vlabTable.meta.client.get_waiter('table_exists').wait(TableName=tableName)

In [2]:
# 若是創建過服務就直接調用
# 調用 SQS
vlabQueues = sqs_client.list_queues( QueueNamePrefix = queueName )
queue_url = vlabQueues['QueueUrls'][0]
print(queue_url)

# 調用 dynamoDB Table
vlabTable = dynamoDB_resource.Table(tableName)

http://sqs.vcloudlab.pro:9324/queue/vcloudlab_sqs_queue


In [ ]:
# 用 flask 建立上傳環境
import os
import datetime
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
UPLOAD_FOLDER = '/home/jovyan/work/pic/'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['SECRET_KEY'] = 'chickenleg'

# 解析檔案名稱
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def show_table():
    
    from flask_table import Table, Col
    from flask import render_template
    
    class ItemTable(Table):
        ID = Col('ID')
        Filename = Col('Filename')
        Url = Col('FileUrl')
        Datetime = Col('Datetime')
    
    ID_MAX = vlabTable.item_count

    class Items(object):
        def __init__(self, ID, Filename, Url, Datetime):
            self.ID = ID
            self.Filename = Filename
            self.Url = Url
            self.Datetime = Datetime

    for item_id in range(1,ID_MAX+1):
        response = vlabTable.get_item(
            Key={
                'id': str(item_id),
            }
        )

        item = response['Item']

        if item_id == 1 :
            items = [Items(item['id'], item['filename'], 
                    item['fileurl'], item['datetime'])]
        else : 
            items += [Items(item['id'], item['filename'], 
                    item['fileurl'], item['datetime'])]

    table = ItemTable(items,border="1")

    return render_template('table.html',table=table)


# 建立轉址位址
from flask import send_from_directory

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)
@app.route('/show', methods=['GET', 'POST'])
def show():
#    if request.method == 'POST':       
    if request.form.get('back') == "back":
        return redirect(url_for('upload_file'))
    return show_table()

# 建立上傳網址
@app.route('/', methods=['GET', 'POST'])
def upload_file():

    if request.method == 'POST':
        
        if request.form.get('Upload') == "Upload":
            
            file = request.files['file']
            
            if file.filename == '':
                flash('No selected file')
                return redirect(request.url)

            if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                                # 儲存在 jupyter 本地端
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                                        # 上傳到 S3
                s3_resource.meta.client.upload_file(
                    '/home/jovyan/work/pic/'+str(filename), 
                    bucketName, 
                    str(filename)
                )

                enqueue_response = sqs_client.send_message(
                    QueueUrl = queue_url, 
                    MessageBody = filename
                )
                print('Message ID : ',enqueue_response['MessageId'])
                
                                            # 透過 SQS  將 filename 抓取
                while True:
                    messages = sqs_client.receive_message(
                      QueueUrl = queue_url,
                      MaxNumberOfMessages = 10
                    ) 
                    if 'Messages' in messages: 
                        for message in messages['Messages']: # 'Messages' is a list
                            # process the messages
                            print(message['Body'])

                            # 先判斷目前的 id
                            getId = vlabTable.item_count
                            setId = getId+1

                            # 將取得的 filename 存入 dynamodb
                            time = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
                            vlabTable.put_item(
                               Item = {
                                   'id': str(setId),
                                   'datetime': str(time),
                                   'fileurl': str(baseUrl)+str(filename),
                                   'filename': str(filename),
                               }
                            )

                            # 將在SQS佇列的Message刪除
                            sqs_client.delete_message(
                              QueueUrl = queue_url,
                              ReceiptHandle=message['ReceiptHandle']
                            )
                    else:
                        print('Queue is now empty')
                        break


                return "上傳成功"

        elif request.form.get('show', None) == "show":
            return redirect(url_for('show'))

        else :
            return "上傳失敗"
            


    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload name=Upload>
    </form>
    <br>
    <br>
    <h1>Show DynamoDB Table</h1>
    <form method=post>
      <input type=submit value=show name=show>
    </form>
    '''
app.run(host='0.0.0.0')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.9.1 - - [30/Jan/2019 07:14:10] "GET / HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:11] "POST / HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:11] "GET /show HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:13] "POST /show HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:13] "GET / HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:13] "POST / HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:14] "GET /show HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:16] "POST /show HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:16] "GET / HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:18] "POST / HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:18] "GET /show HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:14:19] "POST /show HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:14:19] "GET / HTTP/1.1" 200 -
192.168.9.1 - - [30/Jan/2019 07:16:48] "POST / HTTP/1.1" 302 -
192.168.9.1 - - [30/Jan/2019 07:16:49] "GET

In [9]:
print(vlabTable)
print(vlabTable.creation_date_time)



dynamodb.Table(name='vcloudlab_picture')
2019-01-30 06:11:51.115000+00:00


In [ ]:
#fake data
getId = vlabTable.item_count
setId = getId+1
vlabTable.put_item(
    Item = {
        'id': str(setId),
        'datetime': '2019-01-29',
        'fileurl': 'www.google.com',
        'filename': 'Fake.png',
    }
)

In [8]:
show_table()

AttributeError: 'NoneType' object has no attribute 'app'

In [10]:
# Getting an Item
ID_MAX = vlabTable.item_count

response = vlabTable.get_item(
    Key={
        'id': str(1),
    }
)
item = response['Item']
item_id = response['Item']['id']
item_fileurl = response['Item']['fileurl']
item_filename = response['Item']['filename']
item_datetime = response['Item']['datetime']
print('MAX_ID :',ID_MAX)
print(item['filename'])
print(item_id)
print(vlabTable.item_count)

MAX_ID : 3
104.jpg
1
3


In [11]:
for item_id in range(1,ID_MAX+1):
    response = vlabTable.get_item(
    Key={
        'id': str(item_id),
    }
    )
    print('ID : '+response['Item']['id']+', url : '+response['Item']['fileurl']+', NAME : '+response['Item']['filename'])

ID : 1, url : http://s3.vcloudlab.pro:4569/s3/vcloudlab_bucket/104.jpg, NAME : 104.jpg
ID : 2, url : http://s3.vcloudlab.pro:4569/s3/vcloudlab_bucket/bandicam_2018-12-13_14-57-07-161.jpg, NAME : bandicam_2018-12-13_14-57-07-161.jpg
ID : 3, url : http://s3.vcloudlab.pro:4569/s3/vcloudlab_bucket/bandicam_2018-12-13_14-51-44-989.jpg, NAME : bandicam_2018-12-13_14-51-44-989.jpg


In [ ]:
# import things

#pip install flask_table
from flask_table import Table, Col
import os


# Declare your table
class ItemTable(Table):
    ID = Col('ID')
    Filename = Col('Filename')
    Url = Col('FileUrl')
    Datetime = Col('Datetime')

# Get some objects
class Items(object):
    def __init__(self, ID, Filename, Url, Datetime):
        self.ID = ID
        self.Filename = Filename
        self.Url = Url
        self.Datetime = Datetime
        
        
for item_id in range(1,ID_MAX+1):
    response = vlabTable.get_item(
        Key={
            'id': str(item_id),
        }
    )

    item = response['Item']
#         item_id = response['Item']['id']
#         item_fileurl = response['Item']['fileurl']
#         item_filename = response['Item']['filename']
#         item_datetime = response['Item']['datetime']

    if item_id == 1 :
        items = [Items(item['id'], item['filename'], 
                item['fileurl'], item['datetime'])]
    else : 
        items += [Items(item['id'], item['filename'], 
                item['fileurl'], item['datetime'])]
#          Item('Name2', 'Description2'),
#          Item('Name3', 'Description3')]

#         items = [Items(item_id, item_filename, 
#               item_fileurl, item_datetime)]
# Or, equivalently, some dicts

# items = [dict(name='Name1', description='Description1'),
#          dict(name='Name2', description='Description2'),
#          dict(name='Name3', description='Description3')]

# Or, more likely, load items from your database with something like
# items = ItemModel.query.all()

# Populate the table
table = ItemTable(items)
#print(item_id,ID_MAX)

# Print the html

missionFile = open('table.html','w')
missionFile.write(table.__html__())
missionFile.close()   

print(table.__html__())
# or just {{ table }} from within a Jinja template

In [ ]:
for item_id in range(0,ID_MAX):
    print(<table>
<thead><tr><th>Name</th><th>Description</th></tr></thead>
<tbody>

In [ ]:
from flask_table import Table, Col
 
class Results(vlabTable):
    Id = Col(response['Item']['id'])
    Filename = Col(response['Item']['filename'])
    Url = Col(response['Item']['fileurl'])
    Datetime= Col(response['Item']['datetime'])



In [ ]:
from flask_table import Table, Col
 
class Results(vlabTable):
    ID = Col('id')
    Datatime = Col('datatime')
    FileUrl = Col('fileurl')
    Filename = Col('filename')


In [ ]:
for item_id in range(0,5):
    print(str(item_id))
    